# Project: Win 100 points on the Dax

In [143]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Data

In [111]:
dax_symbol = '^GDAXI'

# Yahoo Finance
# dax = yf.Ticker(dax_symbol)
# dax_data = dax.history(period="max")

# Flat CSV file
dax_data = pd.read_csv(filepath_or_buffer=dax_symbol+'.csv')

In [112]:
dax_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0.0
1,1987-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2,1988-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0.0
4,1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0.0
...,...,...,...,...,...,...,...
8806,2022-03-10,13819.440430,13829.969727,13341.790039,13442.099609,13442.099609,148856700.0
8807,2022-03-11,13479.690430,13943.969727,13407.969727,13628.110352,13628.110352,130836100.0
8808,2022-03-14,13827.219727,14082.059570,13815.799805,13929.110352,13929.110352,118949300.0
8809,2022-03-15,13768.440430,13974.839844,13577.809570,13917.269531,13917.269531,101279000.0


In [113]:
fig = go.Figure(data=[go.Candlestick(x=dax_data.index,
                                     open=dax_data.Open,
                                     high=dax_data.High,
                                     low=dax_data.Low,
                                     close=dax_data.Close)])

fig.show()

## Kresten's Strategy

In [117]:
def get_sma(prices, rate):
    return prices.rolling(rate).mean()

def get_bollinger_bands(prices, rate=7):
    sma = get_sma(prices, rate)
    std = prices.rolling(rate).std()
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    return bollinger_up, bollinger_down

In [118]:
# Calculate support and resistance
def get_support_resistance(data):

    _df = data.copy()
    
    _df['Pivot'] = (_df.High + _df.Low + _df.Close) / 3
    _df['R1'] = 2 * _df.Pivot - _df.Low
    _df['S1'] = 2 * _df.Pivot - _df.High  
    _df['R2'] = _df.Pivot + (_df.R1 - _df.S1)  
    _df['S2'] = _df.Pivot - (_df.R1 - _df.S1)  
    _df['R3'] = _df.High + 2 * (_df.Pivot - _df.Low)
    _df['S3'] = _df.Low - 2 * (_df.High - _df.Pivot)

    return _df


In [119]:
# calculate second derivative

def get_derivative(data, col):
    _df = data.copy()
    derivative = (_df[col].shift(-1) - _df[col].shift(1))/2
    return derivative

In [135]:
def estimate_volatility(prices, l):
    """Create an exponential moving average model of the volatility of a stock
    price, and return the most recent (last) volatility estimate.
    
    Parameters
    ----------
    prices : pandas.Series
        A series of adjusted closing prices for a stock.
        
    l : float
        The 'lambda' parameter of the exponential moving average model. Making
        this value smaller will cause the model to weight older terms less 
        relative to more recent terms.
        
    Returns
    -------
    last_vol : float
        The last element of your exponential moving averge volatility model series.
    
    """
    cp = prices.Close.copy()
    alpha = 1-l 
    daily_return = np.power(np.log(cp / cp.shift(1)), 2)
    return_emw = np.sqrt(daily_return.ewm(alpha=alpha, ignore_na=False).mean())
    # last_vol = return_emw[-1]
    return return_emw

In [163]:
def get_daily_price_diff(prices, N):

    _df = prices.copy()

    diff = _df.Close - _df.Close.shift(N)

    return diff

## Compute features

In [ ]:
# get Date as feature column
_df = dax_data.reset_index()

In [127]:
# Date cutoff
_df = _df.loc[_df.Date >= '2020-01-01']

In [131]:
# calculate Bollinger bands
bollinger_up, bollinger_down = get_bollinger_bands(prices=_df['Close'], rate=20)
_df['boll_up'] = bollinger_up
_df['boll_down'] = bollinger_down

In [ ]:
# calculate support and resistance
_df = get_support_resistance(_df)

In [132]:
# calculate derivatives
_df['Derivative'] = get_derivative(_df, col='Close')
_df['Second_derivative'] = get_derivative(_df, col='Derivative')

In [136]:
# calculate volatility
_df['Volatility'] = estimate_volatility(_df, 0.9)

In [147]:
# calculate daily diff
_df['daily_diff'] = get_daily_price_diff(prices=_df, N=1)

In [168]:
# calculate 2-day diff
_df['2day_diff'] = get_daily_price_diff(prices=_df, N=2)

# Plot

In [151]:
# plot DAx with Bolllinger bands

fig = go.Figure()

fig.add_trace(go.Scatter(x=_df.Date, y=_df.Close,
                         mode='lines',
                         name='DAX'))

fig.add_trace(go.Scatter(x=_df.Date, y=_df.boll_up,
                         mode='lines',
                         name='DAX Up'))

fig.add_trace(go.Scatter(x=_df.Date, y=_df.boll_down,
                         mode='lines',
                         name='DAX Down'))

# Add figure title
fig.update_layout(title_text="Historical Dax")

fig.show()

In [156]:
fig = go.Figure()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Bar(x=_df.Date, y=_df.daily_diff,
                         name='Daily Diff'))
fig.add_trace(go.Scatter(x=_df.Date, y=_df.Volatility, 
                         mode='lines', 
                         name="Volatility"), secondary_y=True)

# Set y-axes titles
fig.update_yaxes(title_text="<b>Daily Diff</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Volatility</b>", secondary_y=True)

# Add figure title
fig.update_layout(title_text="Daily Diff vs. Volatility")

fig.show()

# Analysis

In [153]:
mask_date = (_df.Date >= '2021-01-01') & (_df.Date < '2022-01-01')
mask = (_df.Date >= '2021-01-01') & (_df.Date < '2022-01-01') & (_df.daily_diff >= 150)

print(f'Number of days with daily diff greater than 150: {_df.loc[mask].shape[0]} days')

Number of days with daily diff greater than 150: 31 days


In [172]:
mask_date = (_df.Date >= '2021-01-01') & (_df.Date < '2022-01-01')
mask = (_df.Date >= '2021-01-01') & (_df.Date < '2022-01-01') & (_df['2day_diff'] >= 150)

print(f'Number of days with daily diff greater than 150: {_df.loc[mask].shape[0]} days')

Number of days with daily diff greater than 150: 51 days


In [157]:
# plot the distribution of the daily diff

fig = go.Figure()

fig.add_trace(go.Histogram(x=_df.daily_diff))

In [160]:
print(f'Daily differences average: {_df.daily_diff.mean()}')

Daily differences average: 1.8802327023172916


In [ ]:
# Determine the buy and sell signals